In [14]:
import os
import pandas as pd

# Cambiar el directorio de trabajo
os.chdir(r'd:\USER_X\Documents\Trabajos de Git\Proyecto-de-Progra-II')

df = pd.read_csv('cleaned_weather.csv')
df['date'] = pd.to_datetime(df['date']) #Cambio los datos de la columna 'date' a formato datetime
df.head()


,date,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,...,rho,wv,max. wv,wd,rain,raining,SWDR,PAR,max. PAR,Tlog
0,2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,...,1280.62,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45
1,2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,...,1280.33,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51
2,2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,...,1280.29,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60
3,2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,...,1281.97,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70
4,2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,...,1282.08,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81


In [ ]:
#Clase RegistroClimatico
from datetime import datetime

class RegistroClimatico:
    def __init__(self, fecha, temperatura, humedad, presion, tdew):
        self.fecha = fecha
        self.temperatura = temperatura
        self.humedad = humedad
        self.presion = presion
        self.tdew = tdew
    
    def mostrar_info(self):
        return f"Fecha: {self.fecha}, Temperatura: {self.temperatura}°C, Humedad: {self.humedad}%, Presión: {self.presion} mbar, Punto de rocío: {self.tdew}°C"
        
    

In [7]:
fila = df.iloc[0]
registro = RegistroClimatico(fila['date'], fila['T'], fila['rh'], fila['p'], fila['Tdew'])
registro.mostrar_info()

'Fecha: 2020-01-01 00:10:00, Temperatura: 0.71°C, Humedad: 86.1%, Presión: 1008.89 mbar, Punto de rocío: -1.33°C'

In [37]:
class Analizador:
    def __init__(self):
        self.registros = []
        self.alertas = {}
        
    def agregar_registro(self, registro):
        self.registros.append(registro)
    
    def calcular_promedios(self):
        total_temp = total_hum = total_pres = 0
        n = len(self.registros)
        for r in self.registros:
            total_temp += r.temperatura
            total_hum += r.humedad
            total_pres += r.presion
        return {
            "Temperatura_promedio": f"{round(total_temp / n, 2)} °C",
            "Humedad_promedio": f"{round(total_hum / n, 2)} %",
            "Presion_promedio": f"{round(total_pres / n, 2)} mbar"
        }
        
    def max_min(self):
        temps = [r.temperatura for r in self.registros]
        hums = [r.humedad for r in self.registros]
        presiones = [r.presion for r in self.registros]
        return {
            "Temperatura_MAX": f"{max(temps)} °C",
            "Temperatura_MIN": f"{min(temps)} °C",
            "Humedad_MAX": f"{max(hums)} %",
            "Humedad_MIN": f"{min(hums)} %",
            "Presion_MAX": f"{max(presiones)} mbar",
            "Presion_MIN": f"{min(presiones)} mbar"
        }
    
    def generar_alerta(self, temp_umbral = 30, hum_min = 20, hum_max = 90):
        self.alertas = {"Olas_calor_detectadas": [], "Humedad_extrema_detectada": [], "Posible_rocio": []}
        
        for r in self.registros:
            if r.temperatura >= temp_umbral:
                self.alertas["Olas_calor_detectadas"].append(r.fecha)
            if r.humedad <= hum_min or r.humedad >= hum_max:
                self.alertas["Humedad_extrema_detectada"].append(r.fecha)
            if abs(r.temperatura - r.tdew) <= 1:
                self.alertas["Posible_rocio"].append(r.fecha)
        
        return self.alertas

    

In [38]:
analizador = Analizador()

for i in range(len(df)):
    fila = df.iloc[i]
    registro = RegistroClimatico(fila['date'], fila['T'], fila['rh'], fila['p'], fila['Tdew'])
    analizador.agregar_registro(registro)



In [39]:
promedios = analizador.calcular_promedios()
print("Promedios:", promedios)



Promedios: {'Temperatura_promedio': '10.82 °C', 'Humedad_promedio': '72.49 %', 'Presion_promedio': '989.99 mbar'}


In [40]:
maximos_minimos = analizador.max_min()
print("Máximos y Mínimos:", maximos_minimos)

Máximos y Mínimos: {'Temperatura_MAX': '34.8 °C', 'Temperatura_MIN': '-6.44 °C', 'Humedad_MAX': '100.0 %', 'Humedad_MIN': '21.16 %', 'Presion_MAX': '1020.07 mbar', 'Presion_MIN': '955.58 mbar'}


In [41]:
alertas = analizador.generar_alerta(temp_umbral=30, hum_min=20, hum_max=90)
print("Alertas:", alertas)

Alertas: {'Olas_calor_detectadas': [Timestamp('2020-08-01 11:30:00'), Timestamp('2020-08-01 11:40:00'), Timestamp('2020-08-01 11:50:00'), Timestamp('2020-08-01 12:00:00'), Timestamp('2020-08-01 12:10:00'), Timestamp('2020-08-01 12:20:00'), Timestamp('2020-08-01 12:30:00'), Timestamp('2020-08-01 12:40:00'), Timestamp('2020-08-01 12:50:00'), Timestamp('2020-08-01 13:00:00'), Timestamp('2020-08-01 13:10:00'), Timestamp('2020-08-01 13:20:00'), Timestamp('2020-08-01 13:30:00'), Timestamp('2020-08-01 13:40:00'), Timestamp('2020-08-01 13:50:00'), Timestamp('2020-08-01 14:00:00'), Timestamp('2020-08-01 14:10:00'), Timestamp('2020-08-01 14:20:00'), Timestamp('2020-08-01 14:30:00'), Timestamp('2020-08-01 14:40:00'), Timestamp('2020-08-01 14:50:00'), Timestamp('2020-08-01 15:00:00'), Timestamp('2020-08-01 15:10:00'), Timestamp('2020-08-01 15:20:00'), Timestamp('2020-08-01 15:30:00'), Timestamp('2020-08-01 15:40:00'), Timestamp('2020-08-01 15:50:00'), Timestamp('2020-08-01 16:00:00'), Timestamp('2